# Assignment - Knock Aiport Weather Data

### Authored by: Stephen Kerr

This assingment is broken down into 2 sections to Plot the data: 

Part 1
- The temperature
- The mean temperature each day
- The mean temperature for each month

&

Part 2
- The Windspeed (there is data missing from this column)
- The rolling windspeed (say over 24 hours)
- The max windspeed for each day
- The monthly mean of the daily max windspeeds (yer I am being nasty here)


This Notebook will be divided into 2 Parts to address each assessment part seperately. But first we will do imports, load the data, explore and clean the data to be ready for analysis.  

The Weather data analysis was gathered from the Irish Storm Centre from **Knock Airport Weather Station**.  
See their website [here](https://cli.fusio.net/cli/stormcenter/index.html).  

### Imports

In [13]:
# Imports 

import pandas as pd 

import numpy as np

import matplotlib.pyplot as plt

import os

### Load the data

In [16]:
# Load the data
# first 23 rows are metadata

url = "https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv"
df = pd.read_csv(url, skiprows=23, index_col=0)

# output data dir path
output_data_dir = "./assignment06-output-data-assignment06/"

os.makedirs(output_data_dir, exist_ok=True)

df.to_csv(output_data_dir+"raw-weather-data.csv")

C:\Users\kerrs\AppData\Local\Temp\ipykernel_12220\582623101.py:5: DtypeWarning: Columns (2,10,12,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, skiprows=23, index_col=0)


## Dataset Columns 

date:  -  Date and Time (utc)  
rain:  -  Precipitation Amount (mm)  
temp:  -  Air Temperature (C)  
wetb:  -  Wet Bulb Temperature (C)  
dewpt: -  Dew Point Temperature (C)              
rhum:  -  Relative Humidity (%)  
vappr: -  Vapour Pressure (hPa)  
msl:   -  Mean Sea Level Pressure (hPa)  
wdsp:  -  Mean Wind Speed (knot)  
wddir: -  Predominant Wind Direction (degree)  
ww:    -  Synop code for Present Weather  
w:     -  Synop code for Past Weather  
sun:   -  Sunshine duration (hours)  
vis:   -  Visibility (m)  
clht:  -  Cloud height (100's of ft) - 999 if none   
clamt: -  Cloud amount  
ind:   -  Indicator  

In [17]:
# explore the data
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256434 entries, 0 to 256433
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    256434 non-null  object 
 1   ind     256434 non-null  int64  
 2   rain    256434 non-null  object 
 3   ind.1   256434 non-null  int64  
 4   temp    256434 non-null  float64
 5   ind.2   256434 non-null  int64  
 6   wetb    256434 non-null  float64
 7   dewpt   256434 non-null  float64
 8   vappr   256434 non-null  float64
 9   rhum    256434 non-null  int64  
 10  msl     256434 non-null  object 
 11  ind.3   256434 non-null  int64  
 12  wdsp    256434 non-null  object 
 13  ind.4   256434 non-null  int64  
 14  wddir   256434 non-null  object 
 15  ww      256434 non-null  object 
 16  w       256434 non-null  object 
 17  sun     256434 non-null  float64
 18  vis     256434 non-null  object 
 19  clht    256434 non-null  object 
 20  clamt   256434 non-null  object 
dtypes: float64

,ind,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,ind.3,ind.4,sun
count,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000,256434.000000
mean,0.461140,0.020851,8.801566,0.033014,7.843641,6.739962,10.228079,87.915791,2.001396,2.011843,0.122068
std,0.980098,0.143214,4.619329,0.226008,4.223915,4.383908,3.019633,12.191249,0.092992,0.172257,0.360944
min,0.000000,0.000000,-6.900000,0.000000,-7.300000,-11.400000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,5.400000,0.000000,4.700000,3.500000,7.800000,82.000000,2.000000,2.000000,0.000000
50%,0.000000,0.000000,8.900000,0.000000,8.100000,7.000000,10.000000,92.000000,2.000000,2.000000,0.000000
75%,0.000000,0.000000,12.100000,0.000000,11.000000,10.000000,12.300000,97.000000,2.000000,2.000000,0.000000
max,8.000000,2.000000,28.600000,6.000000,23.800000,22.500000,27.200000,100.000000,7.000000,7.000000,4.900000


In [11]:
## check for missing values or null values
#weather_data.isnull().sum()
(weather_data == 0).sum()

date          0
ind      207712
rain     165761
ind.1    251099
temp        459
ind.2    248959
wetb        902
dewpt      1029
vappr        21
rhum         21
msl           0
ind.3        88
wdsp        131
ind.4        92
wddir       109
ww         6469
w          7716
sun      206943
vis           0
clht      66597
clamt      8167
dtype: int64

## Part 1

## Part 2

## End